# OpenDAP via xpublish

Let's first start out with exploring how to convert an xarray dataset into an [opendap_protocol](https://opendap-protocol.readthedocs.io/en/latest/usage.html) dataset.

## xarray -> opendap_protocol

In [1]:
import numpy as np
import opendap_protocol as dap
import xarray as xr

In [2]:
ds = xr.open_dataset("../datasets/ww3_72_east_coast_2022041112.nc")
ds

<xarray.Dataset>
Dimensions:                  (longitude: 381, latitude: 351, time: 73,
                              forecast_reference_time: 1)
Coordinates:
  * longitude                (longitude) float32 -93.0 -92.9 ... -55.1 -55.0
  * latitude                 (latitude) float32 20.0 20.1 20.2 ... 54.9 55.0
  * time                     (time) datetime64[ns] 2022-04-11T12:00:00 ... 20...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2022-04...
Data variables: (12/21)
    hs                       (forecast_reference_time, time, latitude, longitude) float32 ...
    t02                      (forecast_reference_time, time, latitude, longitude) float32 ...
    t0m1                     (forecast_reference_time, time, latitude, longitude) float32 ...
    t01                      (forecast_reference_time, time, latitude, longitude) float32 ...
    fp                       (forecast_reference_time, time, latitude, longitude) float32 ...
    dir                      (forecast_reference_time, time, latitude, longitude) float32 ...
    ...                       ...
    pdir2                    (forecast_reference_time, time, latitude, longitude) float32 ...
    pws0                     (forecast_reference_time, time, latitude, longitude) float32 ...
    pws1                     (forecast_reference_time, time, latitude, longitude) float32 ...
    pws2                     (forecast_reference_time, time, latitude, longitude) float32 ...
    tws                      (forecast_reference_time, time, latitude, longitude) float32 ...
    pnr                      (forecast_reference_time, time, latitude, longitude) float32 ...
Attributes: (12/16)
    WAVEWATCH_III_version_number:     5.16
    WAVEWATCH_III_switches:           F90 SHRD NOGRB NOPA LRB4 NC4 TRKNC PR3 ...
    SIN4 namelist parameter BETAMAX:  1.65
    product_name:                     ww3.20220411.nc
    area:                             NW Atlantic  6 arc min grd2
    latitude_resolution:              0.1000000
    ...                               ...
    easternmost_longitude:            -55.00000
    minimum_altitude:                 -12000 m
    maximum_altitude:                 9000 m
    altitude_resolution:              n/a
    start_date:                       2022-04-11 12:00:00
    stop_date:                        2022-04-11 23:00:00

### Dimensions first

First we need to convert our dimensions into `opendap_protocol.Array`s.

In [4]:
longitude = ds['longitude']
longitude

<xarray.DataArray 'longitude' (longitude: 381)>
array([-93. , -92.9, -92.8, ..., -55.2, -55.1, -55. ], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -93.0 -92.9 -92.8 -92.7 ... -55.2 -55.1 -55.0
Attributes:
    units:          degree_east
    long_name:      longitude
    standard_name:  longitude
    valid_min:      -180.0
    valid_max:      360.0
    axis:           X

In [5]:
longitude.name

'longitude'

In [6]:
longitude.dtype

dtype('float32')

In [10]:
longitude.dtype

dtype('float32')

In [33]:
dtype_dap = {
    np.ubyte: dap.Byte,
    np.int16: dap.Int16,
    np.uint16: dap.UInt16,
    np.int32: dap.Int32,
    np.uint32: dap.UInt32,
    np.float32: dap.Float32,
    np.float64: dap.Float64,
    np.str_: dap.String
}
dtype_dap = {np.dtype(k): v for k, v in dtype_dap.items()}

def dtype_to_daptype(dtype: np.dtype) -> dap.DAPAtom:
    try:
        return dtype_dap[dtype]
    except KeyError:
        print(f"Coercing dtype to string for {dtype}. Probably not a good idea.")
        return dap.String
    
# dtype_dap[longitude.dtype]
dtype_to_daptype(longitude.dtype)

opendap_protocol.protocol.Float32

I probably need to dig into opendap further to figure out the right coercions for datetimes.

In [34]:
dtype_to_daptype(ds['time'].dtype)

Coercing dtype to string for datetime64[ns]. Probably not a good idea.


opendap_protocol.protocol.String

In [24]:
dtype_dap

{dtype('uint8'): opendap_protocol.protocol.Byte,
 dtype('int16'): opendap_protocol.protocol.Int16,
 dtype('uint16'): opendap_protocol.protocol.UInt16,
 dtype('int32'): opendap_protocol.protocol.Int32,
 dtype('uint32'): opendap_protocol.protocol.UInt32,
 dtype('float32'): opendap_protocol.protocol.Float32,
 dtype('float64'): opendap_protocol.protocol.Float64,
 dtype('<U'): opendap_protocol.protocol.String}

In [31]:
def dap_dimension(da: xr.DataArray) -> dap.Array:
    return dap.Array(
        name=da.name,
        data=da.values,
        dtype=dtype_to_daptype(da.dtype)
    )

dap_dimension(longitude)

Lets make a dictionary of our dimensions so that we can find them later.

In [35]:
dims = {}
for dim in ds.dims:
    try:
        dims[dim] = dap_dimension(ds[dim])
    except KeyError as e:
        raise KeyError(f"Problem with dim {dim}: {ds[dim]}") from e
dims

Coercing dtype to string for datetime64[ns]. Probably not a good idea.
Coercing dtype to string for datetime64[ns]. Probably not a good idea.


{'longitude': <opendap_protocol.protocol.Array at 0x18e4623b0>,
 'latitude': <opendap_protocol.protocol.Array at 0x18e462350>,
 'time': <opendap_protocol.protocol.Array at 0x18d77f610>,
 'forecast_reference_time': <opendap_protocol.protocol.Array at 0x18d77e020>}

### Next, the variables

In [36]:
hs = ds['hs']
hs

<xarray.DataArray 'hs' (forecast_reference_time: 1, time: 73, latitude: 351,
                        longitude: 381)>
[9762363 values with dtype=float32]
Coordinates:
  * longitude                (longitude) float32 -93.0 -92.9 ... -55.1 -55.0
  * latitude                 (latitude) float32 20.0 20.1 20.2 ... 54.9 55.0
  * time                     (time) datetime64[ns] 2022-04-11T12:00:00 ... 20...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2022-04...
Attributes:
    long_name:      significant height of wind and swell waves
    standard_name:  sea_surface_wave_significant_height
    globwave_name:  significant_wave_height
    units:          m
    valid_min:      0
    valid_max:      32000

In [37]:
hs.shape

(1, 73, 351, 381)

In [38]:
hs.dtype

dtype('float32')

In [39]:
hs.dims

('forecast_reference_time', 'time', 'latitude', 'longitude')

In [41]:
hs_dap = dap.Grid(
    name=hs.name,
    data=hs.data,
    dtype=dtype_to_daptype(hs.dtype),
    dimensions=[
        dims[dim] for dim in hs.dims
    ]
)
hs_dap

In [45]:
hs_attrs = [dap.Attribute(name=k, value=v, dtype=dap.String) for k, v in hs.attrs.items()]
hs_attrs

In [46]:
hs_dap.append(*hs_attrs)
hs_dap

### Now for a dataset

In [47]:
dataset = dap.Dataset(name="ww3")
dataset.append(*dims.values(), hs_dap)
dataset

In [49]:
print(''.join(dataset.dds()))

Dataset {
    Float32 longitude[longitude = 381];
    Float32 latitude[latitude = 351];
    String time[time = 73];
    String forecast_reference_time[forecast_reference_time = 1];
    Grid {
      Array:
        Float32 hs[forecast_reference_time = 1][time = 73][latitude = 351][longitude = 381];
      Maps:
        String forecast_reference_time[forecast_reference_time = 1];
        String time[time = 73];
        Float32 latitude[latitude = 351];
        Float32 longitude[longitude = 381];
    } hs;
} ww3;



How about a more generalized dtype conversion function. Then in the future we can inspect the dataset further to find out the right conversion.

In [54]:
def dap_dtype(da: xr.DataArray):
    try:
        return dtype_dap[da.dtype]
    except KeyError as e:
        print(f"Unable to match dtype for {da.name}. Going to assume string will work for now...")
        return dap.String

dap_dtype(hs)

opendap_protocol.protocol.Float32

### And lets bundle those all up nicer into functions

In [68]:
def dap_dimension(da: xr.DataArray) -> dap.Array:
    dim = dap.Array(
        name=da.name,
        data=da.values,
        dtype=dap_dtype(da)
    )
    
    for k, v in da.attrs.items():
        dim.append(dap.Attribute(name=k, value=v, dtype=dap.String))
    
    return dim

dap_dimension(ds['longitude'])
dap_dimension(ds['time'])

Unable to match dtype for time. Going to assume string will work for now...


In [59]:
def dap_array(da: xr.DataArray, dims: dict[str, dap.Array]) -> dap.Grid:
    data_array = dap.Grid(
        name=da.name,
        data=da.data,
        dtype=dap_dtype(da),
        dimensions=[
            dims[dim] for dim in da.dims
        ]
    )
    
    for k, v in da.attrs.items():
        data_array.append(dap.Attribute(name=k, value=v, dtype=dap.String))
    
    return data_array

dap_array(ds['hs'], dims)

In [72]:
def dap_dataset(ds: xr.Dataset, name: str) -> dap.Dataset:
    dataset = dap.Dataset(name=name)
    
    dims = {}
    for dim in ds.dims:
        dims[dim] = dap_dimension(ds[dim])
            
    dataset.append(*dims.values())
    
    for var in ds.variables:
        if var not in ds.dims:
            data_array = dap_array(ds[var], dims)
            dataset.append(data_array)
    
    for k, v in ds.attrs.items():
        dataset.append(dap.Attribute(name=k, value=v, dtype=dap.String))
    
    return dataset

ww3_dap = dap_dataset(ds, "ww3")
ww3_dap

Unable to match dtype for time. Going to assume string will work for now...
Unable to match dtype for forecast_reference_time. Going to assume string will work for now...


In [73]:
print("".join(ww3_dap.dds()))

Dataset {
    Float32 longitude[longitude = 381];
    Float32 latitude[latitude = 351];
    String time[time = 73];
    String forecast_reference_time[forecast_reference_time = 1];
    Grid {
      Array:
        Float32 hs[forecast_reference_time = 1][time = 73][latitude = 351][longitude = 381];
      Maps:
        String forecast_reference_time[forecast_reference_time = 1];
        String time[time = 73];
        Float32 latitude[latitude = 351];
        Float32 longitude[longitude = 381];
    } hs;
    Grid {
      Array:
        Float32 t02[forecast_reference_time = 1][time = 73][latitude = 351][longitude = 381];
      Maps:
        String forecast_reference_time[forecast_reference_time = 1];
        String time[time = 73];
        Float32 latitude[latitude = 351];
        Float32 longitude[longitude = 381];
    } t02;
    Grid {
      Array:
        Float32 t0m1[forecast_reference_time = 1][time = 73][latitude = 351][longitude = 381];
      Maps:
        String forecast_reference_

In [74]:
print("".join(ww3_dap.das()))

Attributes {
    longitude {
        String units "degree_east";
        String long_name "longitude";
        String standard_name "longitude";
        String valid_min "-180.0";
        String valid_max "360.0";
        String axis "X";
    }
    latitude {
        String units "degree_north";
        String long_name "latitude";
        String standard_name "latitude";
        String valid_min "-90.0";
        String valid_max "180.0";
        String axis "Y";
    }
    time {
        String long_name "julian day (UT)";
        String standard_name "time";
        String conventions "relative julian days with decimal part (as parts of the day )";
        String axis "T";
    }
    forecast_reference_time {
        String long_name "Forecast Reference Time";
        String standard_name "forecast_reference_time";
        String conventions "relative julian days with decimal part (as parts of the day )";
    }
    hs {
        String long_name "significant height of wind and swell wav

This will break things, but probably for very good reasons (too much data in the stream for Jupyer to handle)!

```py
print(b"".join(dataset.dods()))
```

That seemed to work. Lets move those into a router and give it a shot.

## Loading with OpenDAP

Now we can try loading the dataset via OpenDAP.

In [78]:
ds_from_dap = xr.open_dataset("http://0.0.0.0:9005/datasets/ww3/opendap")
ds_from_dap

IndexError: The indexing operation you are attempting to perform is not valid on netCDF4.Variable object. Try loading your data into memory first by calling .load().

It fails, but it identifies that an index is the issue. We probably don't have the right datatypes on the datetime dimensions, so we at least have a place to start exploring.

In [80]:
forecast_time = ds["forecast_reference_time"]
forecast_time

<xarray.DataArray 'forecast_reference_time' (forecast_reference_time: 1)>
array(['2022-04-11T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2022-04...
Attributes:
    long_name:      Forecast Reference Time
    standard_name:  forecast_reference_time
    conventions:    relative julian days with decimal part (as parts of the d...

In [81]:
forecast_time.dtype

dtype('<M8[ns]')

In [82]:
forecast_time.data

array(['2022-04-11T12:00:00.000000000'], dtype='datetime64[ns]')

In [83]:
forecast_time.data.dtype

dtype('<M8[ns]')

In [84]:
forecast_time.encoding

{'zlib': True,
 'shuffle': True,
 'complevel': 1,
 'fletcher32': False,
 'contiguous': False,
 'chunksizes': (512,),
 'source': '/Users/akerney/GMRI/restful-grids/datasets/ww3_72_east_coast_2022041112.nc',
 'original_shape': (1,),
 'dtype': dtype('float64'),
 '_FillValue': nan,
 'units': 'days since 1990-01-01',
 'calendar': 'proleptic_gregorian'}

In [85]:
forecast_time.encoding['dtype']

dtype('float64')

In [87]:
def dap_dtype(da: xr.DataArray):
    """ Return a DAP type for the xr.DataArray """
    try:
        return dtype_dap[da.encoding["dtype"]]
    except KeyError as e:
        logger.warning(
            f"Unable to match dtype for {da.name}. Going to assume string will work for now... ({e})"
        )
        return dap.String

Changed things to use the encoded dtype, and...

In [86]:
ds_from_dap = xr.open_dataset("http://0.0.0.0:9005/datasets/ww3/opendap")
ds_from_dap

<xarray.Dataset>
Dimensions:                  (longitude: 381, latitude: 351, time: 73,
                              forecast_reference_time: 1)
Coordinates:
  * longitude                (longitude) float32 -93.0 -92.9 ... -55.1 -55.0
  * latitude                 (latitude) float32 20.0 20.1 20.2 ... 54.9 55.0
  * time                     (time) float64 1.65e+18 1.65e+18 ... 1.65e+18
  * forecast_reference_time  (forecast_reference_time) float64 1.65e+18
Data variables: (12/21)
    hs                       (forecast_reference_time, time, latitude, longitude) float32 ...
    t02                      (forecast_reference_time, time, latitude, longitude) float32 ...
    t0m1                     (forecast_reference_time, time, latitude, longitude) float32 ...
    t01                      (forecast_reference_time, time, latitude, longitude) float32 ...
    fp                       (forecast_reference_time, time, latitude, longitude) float32 ...
    dir                      (forecast_reference_time, time, latitude, longitude) float32 ...
    ...                       ...
    pdir2                    (forecast_reference_time, time, latitude, longitude) float32 ...
    pws0                     (forecast_reference_time, time, latitude, longitude) float32 ...
    pws1                     (forecast_reference_time, time, latitude, longitude) float32 ...
    pws2                     (forecast_reference_time, time, latitude, longitude) float32 ...
    tws                      (forecast_reference_time, time, latitude, longitude) float32 ...
    pnr                      (forecast_reference_time, time, latitude, longitude) float32 ...
Attributes: (12/16)
    WAVEWATCH_III_version_number:     5.16
    WAVEWATCH_III_switches:           F90 SHRD NOGRB NOPA LRB4 NC4 TRKNC PR3 ...
    SIN4_namelist_parameter_BETAMAX:  1.649999976158142
    product_name:                     ww3.20220411.nc
    area:                             NW Atlantic  6 arc min grd2
    latitude_resolution:              0.1000000
    ...                               ...
    easternmost_longitude:            -55.00000
    minimum_altitude:                 -12000 m
    maximum_altitude:                 9000 m
    altitude_resolution:              n/a
    start_date:                       2022-04-11 12:00:00
    stop_date:                        2022-04-11 23:00:00

Holy crap, that worked.